In [1]:
# Importation des librairies externes
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data@master --ignore-install PyYAML
!pip install python-Levenshtein
!pip install openpyxl

  Cloning https://github.com/InseeFrLab/Py-Insee-Data (to revision master) to /tmp/pip-req-build-knba8h26
  Running command git clone --filter=blob:none -q https://github.com/InseeFrLab/Py-Insee-Data /tmp/pip-req-build-knba8h26
  Resolved https://github.com/InseeFrLab/Py-Insee-Data to commit 29e47be68f31bac3a8b5874e827ff398ed53d1f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached DateTime-4.3-py2.py3-none-any.whl (60 kB)
  Using cached pandas-1.3.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached p

In [20]:
# Importation des librairies 
import requests
import pandas as pd
import pynsee
import pynsee.download
from pynsee.sirene import *
import time
from tqdm import tqdm
from pynsee.utils.init_conn import init_conn
import os
from zipfile import ZipFile, Path
import io

In [3]:
# token API insee
init_conn(insee_key=os.environ['INSEE_KEY'], insee_secret=os.environ['INSEE_SECRET'])

KeyError: 'INSEE_KEY'

In [11]:
#Téléchargement des données Filosofi au niveau IRIS (millénisme 2018)
URL_FILO_DECL_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DEC_IRIS_2018.xlsx"
URL_FILO_DISP_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DISP_IRIS_2018.xlsx"

donnees_filo_decl_2018=pd.read_excel(URL_FILO_DECL_2018, sheet_name='IRIS_DEC', skiprows=5)
donnees_filo_disp_2018=pd.read_excel(URL_FILO_DISP_2018, sheet_name='IRIS_DISP', skiprows=5)

In [12]:
print(donnees_filo_decl_2018.head(10))
print(donnees_filo_disp_2018.head(10))

        IRIS                           LIBIRIS    COM             LIBCOM  \
0  010040101  Les Perouses-Triangle d'Activité  01004  Ambérieu-en-Bugey   
1  010040102                     Longeray-Gare  01004  Ambérieu-en-Bugey   
2  010040201    Centre-Saint-Germain-Vareilles  01004  Ambérieu-en-Bugey   
3  010040202                 Tiret-Les Allymes  01004  Ambérieu-en-Bugey   
4  010330102                      Centre Ville  01033         Valserhône   
5  010330103             Lancrans-Coupy-Vanchy  01033         Valserhône   
6  010330201        Arc Vouvray-Gare-Châtillon  01033         Valserhône   
7  010330202               Plateau de Musinens  01033         Valserhône   
8  010330301                             Arlod  01033         Valserhône   
9  010330401            Châtillon-en-Michaille  01033         Valserhône   

   DEC_PIMP18  DEC_TP6018  DEC_Q118  DEC_MED18  DEC_Q318  DEC_EQ18  ...  \
0        45.4        31.0   11150.0    18540.0   25900.0      0.80  ...   
1        39.3

In [17]:
#Téléchargement des populations légales du millénisme 2018 
URL_POP_LEG="https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_POP_LEG)

open("ensemble.zip", 'wb').write(r.content)
    
    
with ZipFile("ensemble.zip",'r') as myzip:
    data = myzip.open("Departements.csv")

donnees_pop_leg_18=pd.read_csv(data,sep=';')     

In [18]:
donnees_pop_leg_18.sample(10)

,CODREG,REG,CODDEP,DEP,NBARR,NBCAN,NBCOM,PMUN,PTOT
40,75,Nouvelle-Aquitaine,40,Landes,2,15.0,327,410355,422602
0,84,Auvergne-Rhône-Alpes,01,Ain,4,23.0,393,647634,663720
91,11,Île-de-France,91,Essonne,3,21.0,194,1296641,1311028
3,93,Provence-Alpes-Côte d'Azur,04,Alpes-de-Haute-Provence,4,15.0,198,164068,168613
11,76,Occitanie,12,Aveyron,3,23.0,285,279274,289583
41,24,Centre-Val de Loire,41,Loir-et-Cher,3,15.0,267,330248,338835
63,84,Auvergne-Rhône-Alpes,63,Puy-de-Dôme,5,31.0,464,659048,673671
57,44,Grand Est,57,Moselle,5,27.0,725,1043524,1062362
52,44,Grand Est,52,Haute-Marne,3,17.0,426,174069,179062
2,84,Auvergne-Rhône-Alpes,03,Allier,3,19.0,317,337171,346156


In [21]:
### Téléchargement des données Sirene sur les établissements Siret de boucherie-charcuterie (code 4722Z) à l'instant T
# Téléchargement via Insee sirene data  by sirene.fr website

session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)
donnees_sirene_fr.shape

(20165, 76)

In [22]:
donnees_sirene_fr.sample(10)

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,...,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
4466,425002623,00063,42500262300063,O,2009-01-01,02,2018,4722ZA,2021-02-23T11:47:19,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
9606,523568517,00016,52356851700016,O,2010-06-28,NN,NaN,4722ZA,2021-01-27T14:06:58,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
11462,789418233,00019,78941823300019,O,2012-07-25,01,2019,NaN,2021-10-27T08:57:43,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
12105,795146844,00019,79514684400019,O,2013-10-01,NN,NaN,4722ZB,2019-06-24T13:51:26,true,...,NaN,NaN,A,NaN,NaN,NaN,SARL EL MALTI VIANDES,47.22Z,NAFRev2,O
9619,523766004,00015,52376600400015,O,2010-07-01,01,2019,4722ZB,2021-10-27T08:31:34,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
13914,814318382,00029,81431838200029,O,2016-02-01,11,2019,NaN,2021-10-27T09:21:09,false,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
10456,535097521,00012,53509752100012,O,2011-10-03,NaN,NaN,4722ZB,2019-11-14T14:00:37,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
9919,529327579,00023,52932757900023,O,2011-03-01,02,2019,4722ZB,2021-10-27T08:36:13,false,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
15415,829251412,00021,82925141200021,O,2018-06-01,00,2019,4722ZB,2021-10-27T09:33:29,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
8775,511779688,00031,51177968800031,O,2018-10-15,NaN,NaN,4722ZB,2019-04-03T08:49:20,false,...,NaN,NaN,A,MON BOUCHER-TRAITEUR SAGA,NaN,NaN,NaN,47.22Z,NAFRev2,O


In [26]:
#Téléchargement des données de crimes et délits rapportés déclarés auprès des commissariats 
url_datagouv_min_interieur="https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520"

df_pn2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services PN 2018', skiprows=[1,2])
df_gn2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services GN 2018', skiprows=[1,2])


In [27]:
df_gn2018.head(10)
df_pn2018.head(10)

,Année 2018 - services de police,Départements,01,01.1,01.2,01.3,02,02.1,02.2,02.3,...,974.6,976,976.1,976.2,987,987.1,987.2,988,988.1,988.2
0,1,Règlements de compte entre malfaireurs,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Homicides pour d'autres motifs,0,0,2,1,2,1,0,1,...,2,0,0,0,0,0,1,0,0,3
3,4,Tentatives d'homicides pour voler et à l'occas...,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
4,5,Tentatives homicides pour d'autres motifs,0,0,1,0,0,0,4,1,...,9,0,0,2,0,0,0,0,0,2
5,6,Coups et blessures volontaires suivis de mort,0,0,1,0,0,0,0,1,...,0,0,0,2,0,0,0,0,0,2
6,7,Autres coups et blessures volontaires criminel...,0,0,236,135,102,193,223,430,...,870,1,1,480,1,1,401,1,0,972
7,8,Prises d'otages à l'occasion de vols,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,Prises d'otages dans un autre but,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,10,Sequestrations,0,0,9,3,3,1,4,11,...,8,1,0,12,0,0,1,0,0,6


In [29]:
#Téléchargement des données d'emploi localisées
emploi_statut_18 = pynsee.download.telechargerDonnees("ESTEL_T201", date = "2018")
emploi_statut_18.sample(10)

emploi_sexe_18 = pynsee.download.telechargerDonnees("ESTEL_T202", date = "2018")
emploi_sexe_18.sample(10)

Data will be stored in the following location: /tmp/tmpdjdcac_k


Downloading: 100%|██████████| 2.26M/2.26M [00:00<00:00, 2.66MiB/s]


No destination directory defined. Data have been written there: /tmp/tmpy_5_oceg


Extracting: 100%|██████████| 13.6M/13.6M [00:00<00:00, 213MB/s]


Data will be stored in the following location: /tmp/tmp8bzsbool


Downloading: 100%|██████████| 2.26M/2.26M [00:01<00:00, 1.24MiB/s]


No destination directory defined. Data have been written there: /tmp/tmpyxqtd6ud


Extracting: 100%|██████████| 13.6M/13.6M [00:00<00:00, 246MB/s]
/opt/conda/lib/python3.9/site-packages/pynsee/download/__init__.py:73: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  return chargerDonnees(


,NA38,GEO,SEXE,ANNEE2,EFFSAL
450228,TBEC5,F93,E,2005,75565
482058,TGUJZJA,F94,E,1995,309
149614,TGUJZJC,F2861,F,2000,31
58929,TGUJZJC,F1195,E,2012,7635
386567,TGU,F7681,H,1994,17413
128709,TGURU,F2790,E,1992,1457
136109,TGUGZ,F2814,H,1998,16227
413429,TGUMN,F8415,H,2008,863
112433,TAZ,F2770,H,1996,838
124532,TGURURZ,F2789,H,2009,512
